In [37]:
import jax
import jax.numpy as jnp

import genjax


@genjax.gen
def model(μ):
    x = genjax.normal(μ, 1.0) @ "x"
    return (x - μ) ** 2

In [38]:
key = jax.random.PRNGKey(314159)
key, sub_keys = genjax.slash(key, 2000)
adev_prog = genjax.adev.lang(model)
_, (v,), (tangents,) = jax.vmap(adev_prog.grad_estimate, in_axes=(0, None, None))(
    sub_keys, (2.0,), (1.0,)
)

In [39]:
jnp.mean(tangents)

In [40]:
jnp.mean(v)